<a href="https://colab.research.google.com/github/Yahnavi/YouTube_Sentiment_Analysis/blob/main/Youtube_SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install emoji
!pip install vaderSentiment
!pip install goolge-api-python-client
!pip install --upgrade gradio

In [61]:
from googleapiclient.discovery import build
import re
import emoji
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt
import gradio as gr


def sentiment_analysis(url):
  API_KEY = 'AIxxxxxxxxxxxxxx'
  youtube = build('youtube','v3', developerKey=API_KEY)
  video_id = url[-11:]

  video_response = youtube.videos().list(
    part = 'snippet',
    id = video_id
   ).execute()

  video_snippet = video_response['items'][0]['snippet']
  uploader_channel_id = video_snippet['channelId']

  comments = []
  nextPageToken = None
  while (len(comments)) < 600:
    request = youtube.commentThreads().list(
      part = 'snippet',
      videoId = video_id,
      maxResults = 100,
      pageToken = nextPageToken
  )
    response = request.execute()
    for item in response['items']:
      comment = item['snippet']['topLevelComment']['snippet']
      if comment['authorChannelId']['value'] != uploader_channel_id:
        comments.append(comment['textDisplay'])
    nextPageToken = response.get('nextPageToken')

    if not nextPageToken:
      break

  hyperlink_pattern = re.compile(
    r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')

  threshold_ratio = 0.65
  relevant_comments = []

  for comment_text in comments:

    comment_text = comment_text.lower().strip()
    emojis = emoji.emoji_count(comment_text)
    text_characters = len(re.sub(r'\s','',comment_text))

    if(any(char.isalnum() for char in comment_text) and not hyperlink_pattern.search(comment_text)):
      if emojis == 0 or (text_characters / (text_characters + emojis)) > threshold_ratio:
        relevant_comments.append(comment_text)

  f = open("ytcomments.txt",'w',encoding='utf-8')
  for idx,comment in enumerate(relevant_comments):
     f.write(str(comment)+"\n")
  f.close()

  def sentiment_scores(comment,polarity):
    sentiment_object = SentimentIntensityAnalyzer()
    sentiment_dict = sentiment_object.polarity_scores(comment)
    polarity.append(sentiment_dict['compound'])
    return polarity

  polarity = []
  positive_comments = []
  negative_comments = []
  neutral_comments = []

  f = open("ytcomments.txt",'r',encoding='`utf-8')
  comments = f.readlines()
  f.close()

  # print("Analyzing comments...")
  for index,items in enumerate(comments):
    polarity = sentiment_scores(items,polarity)

    if polarity[-1] > 0.05:
      positive_comments.append(items)
    elif polarity[-1] < -0.05:
      negative_comments.append(items)
    else:
      neutral_comments.append(items)

  # avg_polarity = sum(polarity)/len(polarity)
  # print("Average Polarity:",avg_polarity)
  # if avg_polarity > 0.05:
  #   print("The Video has got a Positive response")
  # elif avg_polarity < -0.05:
  #   print("The Video has got a Negative response")
  # else:
  #   print("The Video has got a Neutral response")

  # print("The comment with most positive sentiment:", comments[polarity.index(max(
  #     polarity))],"with score",max(polarity),"and length",len(comments[polarity.index(max(polarity))]))
  # print("The comment with most negative sentiment:",comments[polarity.index(min(
  #   polarity))],"with score",min(polarity),"and length",len(comments[polarity.index(min(polarity))]))

  positive_count = len(positive_comments)
  negative_count = len(negative_comments)
  neutral_count = len(neutral_comments)

  labels = ['Positive', 'Negative', 'Neutral']
  sizes = [int(positive_count), int(negative_count), int(neutral_count)]
  colors = ['gold', 'lightcoral', 'lightskyblue']


  plt.figure(figsize=(8, 8))
  plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=140)
  plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

  html_string = """
    <style>
      .positive {
          background-color: lightgreen;
          padding: 10px;
          margin: 10px;
      }
      .negative {
          background-color: lightcoral;
          padding: 10px;
          margin: 10px;
      }
      .neutral {
          background-color: lightgrey;
          padding: 10px;
          margin: 10px;
      }
      .centre{
        text-align: center;
        font-weight: bold;
        font-size: 20px;
        display: inline;
        padding: 10px;
        margin: 10px 40px 20px 60px;
        border-radius: 10px;
        border:none;

      }
      </style>
      """

  count_string = f"<div class='positive centre' >Positive: {positive_count}</div>  <div class='negative centre'>Negative: {negative_count}</div>  <div class='neutral centre'>Neutral: {neutral_count}</div>"


  for com in comments:
     if com in positive_comments:
        html_string += f"<div class='positive'>{com}</div>"
     elif com in negative_comments:
        html_string += f"<div class='negative'>{com}</div>"
     else:
        html_string += f"<div class='neutral'>{com}</div>"

  plt.savefig('pie_chart.png')
  plt.close()
  return [count_string,'pie_chart.png',html_string]


css = """

.gradio-container {
  background: url('https://e0.pxfuel.com/wallpapers/259/652/desktop-wallpaper-youtube-logo-light-art-youtube-emblem-blue-light-line-background-youtube-neon-logo-creative-art-youtube.jpg');
  background-repeat: no-repeat;
  background-size: cover;
  background-attachment: fixed;
  background-position: center;
  }
  h1{
  color:white;
  font-family:Arial, Helvetica, sans-serif;
}
"""


demo = gr.Interface(fn=sentiment_analysis,
                    inputs=gr.Textbox(label="URL", placeholder="Enter the video URL"),
                    outputs=["html","image","html"],
                    title="Youtube video analysis",
                    css = css )

demo.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://2fab5df0afa9722b5a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7862 <> https://2fab5df0afa9722b5a.gradio.live
